In [1]:
import pandas as pd
import numpy as np

In [2]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'


In [3]:
ratio_actividad = pd.read_csv(pathdata + 'data_output/indicadores/01_ratio_actividad.csv', index_col=0)
empleo_asalariado_con_descuento = pd.read_csv(pathdata + 'data_output/indicadores/02_empleo_asalariado_con_descuento.csv', index_col=0)
horas_remuneradas_media = pd.read_csv(pathdata + 'data_output/indicadores/03_horas_promedio_ocup_ppal.csv', index_col=0)
ingreso_salarial_media = pd.read_csv(pathdata + 'data_output/indicadores/04_ingreso_salarial.csv', index_col=0)
ratio_minutos_promedio_no_pago = pd.read_csv(pathdata + 'data_output/indicadores/05_tiempo_no_remunerado.csv', index_col=0)
inactivos_con_ingreso = pd.read_csv(pathdata + 'data_output/indicadores/06_dependencia_economica.csv', index_col=0)
hogares_jefatura = pd.read_csv(pathdata + 'data_output/indicadores/07_hogares_no-pobres_jefatura.csv', index_col=0)

In [4]:
ratio_actividad = ratio_actividad.sort_index()
empleo_asalariado_con_descuento = empleo_asalariado_con_descuento.sort_index()
horas_remuneradas_media = horas_remuneradas_media.sort_index()
ingreso_salarial_media = ingreso_salarial_media.sort_index()
ratio_minutos_promedio_no_pago = ratio_minutos_promedio_no_pago.sort_index()
inactivos_con_ingreso = inactivos_con_ingreso.sort_index()
hogares_jefatura = hogares_jefatura.sort_index()


In [5]:
variables = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media, ingreso_salarial_media, ratio_minutos_promedio_no_pago, inactivos_con_ingreso, hogares_jefatura]

indices_ordenados = True

# Compara los índices de los DataFrames en un bucle
for i in range(len(variables) - 1):
    if not variables[i].index.equals(variables[i + 1].index):
        indices_ordenados = False
        break

# Imprime el resultado
print("Los índices están ordenados de la misma manera:", indices_ordenados)

Los índices están ordenados de la misma manera: True


In [6]:
variables = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media, ingreso_salarial_media, ratio_minutos_promedio_no_pago, inactivos_con_ingreso, hogares_jefatura]
columnas = ['Actividad', 'Formalidad', 'Horas promedio remuneradas', 'Ingreso laboral promedio', 'Tiempo TNR', 'Dependencia', 'No Pobreza']

for columna, variable in zip(columnas,variables):
    variable[columna] = np.where(variable[columna] > 100, 100, variable[columna])
    variable['Ranking'] = variable[columna].rank(ascending=False, method='first')


In [7]:
componente_insercion = (ratio_actividad['Actividad'] + \
                        empleo_asalariado_con_descuento['Formalidad'] + \
                        horas_remuneradas_media['Horas promedio remuneradas'] + \
                        ingreso_salarial_media['Ingreso laboral promedio'])/4

componente_insercion = componente_insercion.to_frame()
componente_insercion.columns = ['Insercion laboral']
componente_insercion['Ranking'] = componente_insercion['Insercion laboral'].rank(ascending=False, method='first')

In [8]:
componente_autonomia_economica = (ratio_minutos_promedio_no_pago['Tiempo TNR'] + \
                                inactivos_con_ingreso['Dependencia'] + \
                                hogares_jefatura['No Pobreza'])/3

componente_autonomia_economica = componente_autonomia_economica.to_frame()
componente_autonomia_economica.columns = ['Autonomia economica']
componente_autonomia_economica['Ranking'] = componente_autonomia_economica['Autonomia economica'].rank(ascending=False, method='first')

In [9]:
indice_mti = np.sqrt(componente_autonomia_economica['Autonomia economica'] * componente_insercion['Insercion laboral']).to_frame()
indice_mti.columns = ['Indice MTeI']
indice_mti['Ranking'] = indice_mti['Indice MTeI'].rank(ascending=False, method='first')

In [25]:
## Genero Tabla resumen
labels_variables_todas = ['Actividad', 'Formalidad', 'Horas promedio remuneradas', 'Ingreso laboral promedio',
                  'Tiempo TNR', 'Dependencia', 'No Pobreza',
                  'Insercion laboral', 'Autonomia economica', 'Indice MTeI']

columnas = ['Media', 'Desvio estandar', 'Minimo', 'Maximo', 'Rango']

variables_todas = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media, \
                    ingreso_salarial_media, ratio_minutos_promedio_no_pago, inactivos_con_ingreso, \
                        hogares_jefatura, componente_insercion, componente_autonomia_economica, indice_mti]

df_tabla_resumen = pd.DataFrame(index=labels_variables_todas, columns=columnas)


In [26]:

for i, idf in enumerate(variables_todas):
    icolumna = labels_variables_todas[i]
    imedia = idf[icolumna].mean()
    istd = idf[icolumna].std()
    imin = idf[icolumna].min() 
    imax = idf[icolumna].max()
    irango = imax - imin
    df_tabla_resumen.iloc[i,0] = imedia
    df_tabla_resumen.iloc[i,1] = istd
    df_tabla_resumen.iloc[i,2] = imin
    df_tabla_resumen.iloc[i,3] = imax
    df_tabla_resumen.iloc[i,4] = irango

In [31]:
df_tabla_resumen = df_tabla_resumen.astype('float').round(2)

In [34]:
df_tabla_resumen.to_clipboard()

,Media,Desvio estandar,Minimo,Maximo,Rango
Actividad,68.78,7.28,47.83,80.79,32.97
Formalidad,92.60,5.12,84.53,100.00,15.47
Horas promedio remuneradas,77.33,4.34,68.64,87.72,19.08
Ingreso laboral promedio,77.39,7.29,65.82,89.63,23.81
Tiempo TNR,48.97,6.04,40.31,56.54,16.23
Dependencia,90.34,7.22,77.07,100.00,22.93
No Pobreza,89.41,8.02,70.00,100.00,30.00
Insercion laboral,79.03,3.48,73.02,86.00,12.98
Autonomia economica,76.24,3.28,69.86,82.37,12.52
Indice MTeI,77.60,2.82,72.59,83.95,11.36


In [32]:
## Genero tabla output

In [33]:
provincias = indice_mti.index.to_list()
provincia_repetidos = [value for value in provincias for _ in range(7)]

componentes = ['Insercion laboral'] * 4 + ['Autonomia Economica'] * 3
componentes_repetidos = componentes*24

indicadores = ['Actividad', 'Formalidad', 'Horas trabajadas', 'Ingreso salarial',
                  'Tiempo trabajo no remunerado', 'Dependencia economica', 'No-Pobreza']

In [34]:
data = {
    'Provincia': provincia_repetidos,
    'Componente': componentes_repetidos,
    'Indicador': indicadores*24
}

df = pd.DataFrame(data)

multiindex = pd.MultiIndex.from_frame(df[['Provincia', 'Componente', 'Indicador']])
df.set_index(multiindex, inplace=True)
df.drop(columns=['Provincia', 'Componente', 'Indicador'], inplace=True)

df[['Mujer', 'Varon', 'Ratio', 'N pob varon', 'N pob mujer', 'N pob tot', 'N varon', 'N mujer', 'CV', 'Error', 'Ranking indicador', 'Componentes', 'Ranking componente', 'Indice', 'Ranking Indice']] = pd.NaT

In [35]:
for indicador, columna, variable in zip(indicadores, columnas, variables):
    if indicador == 'Tiempo trabajo no remunerado':
        values = variable[['Mujer', 'Varon', columna, 'Ranking']].values
        df.loc[df.index.get_level_values('Indicador') == indicador, ['Mujer', 'Varon', 'Ratio', 'Ranking indicador']] = values

    else:    
        values = variable[['Mujer', 'Varon', columna, 'N_pob_v', 'N_pob_m', 'N_pob_tot', 'N_v', 'N_m', 'ER', 'ME', 'Ranking']].values
        df.loc[df.index.get_level_values('Indicador') == indicador, ['Mujer', 'Varon', 'Ratio', 'N pob varon', 'N pob mujer', 'N pob tot', 'N varon', 'N mujer', 'CV', 'Error', 'Ranking indicador']] = values


In [36]:
componente_insercion.to_csv(pathdata + 'data_output/componentes_e_indice/01_insercion_laboral.csv')
componente_autonomia_economica.to_csv(pathdata + 'data_output/componentes_e_indice/02_insercion_autonomia_economica.csv')
indice_mti.to_csv(pathdata + 'data_output/componentes_e_indice/01_indice_MTeI.csv')

In [66]:
values = componente_insercion.values
df.loc[df.index.get_level_values('Indicador') == 'Actividad', ['Componentes', 'Ranking componente']] = values

In [70]:
values = componente_autonomia_economica.values
df.loc[df.index.get_level_values('Indicador') == 'Tiempo trabajo no remunerado', ['Componentes', 'Ranking componente']] = values

In [73]:
values = indice_mti.values
df.loc[df.index.get_level_values('Indicador') == 'Actividad', ['Indice', 'Ranking Indice']] = values

In [74]:
df.to_csv(pathdata + 'data_output/tablas/01_indicadores_principales.csv')